In [3]:
import pandas as pd
import re
import nltk

import matplotlib.pyplot as plt
%matplotlib inline

In [10]:
data_path = "~/Documents/Data_Science_Projects/Chicago_Tool/Data/"

df1 = pd.read_csv(data_path + "Business_Licenses_-_Current_Active.csv")

/opt/anaconda3/envs/ls_geo_env/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3457: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [30]:
#list(df1.columns)


Multiple licenses for the same business?
YES
The majority seem to be hotels that have distinct licenses. But there are others like JOE'S BAR, which has multiple licenses.

As for building up a text to create features from, I think the business name, license description, and, business activity can be joined for each business name/address.

From there, pick a sample of entries and hand classify them (or rule classify). And then build a model.

I'm most interested in:
* sit down restaurants
* fast food restaurants
* bars
* grocery stores/food markets
* convenience stores
Then to lesser degree
* liquor stores
* currency exchanges

In [34]:
df1.groupby(["DOING BUSINESS AS NAME", "ADDRESS"])["LICENSE DESCRIPTION"].nunique().sort_values(ascending=False).head(30)

DOING BUSINESS AS NAME                                ADDRESS                            
HYATT REGENCY MCCORMICK PLACE HOTEL                   2233 S DR MARTIN LUTHER KING JR DR     7
CHICAGO HILTON & TOWERS                               720 S MICHIGAN AVE                     7
SOHO HOUSE CHICAGO                                    113-125 N GREEN ST BSMT - 6TH          7
HARD ROCK CAFE                                        63 W ONTARIO ST 1ST                    6
CELESTE & DISCO                                       111 W HUBBARD ST 1-4                   6
FROM HERE ON FOOD HALL & MARKET                       433 W VAN BUREN ST 1-2                 6
HOMEWOOD SUITES BY HILTON CHICAGO DOWNTOWN SOUTHLOOP  1101 S WABASH AVE 1-22                 6
THE WIT                                               201 N STATE ST 1-27                    6
REVEL MOTOR ROW                                       2400-2410 S MICHIGAN AVE               6
AC MARRIOTT CHICAGO DOWNTOWN                          6

In [36]:
df[df["DOING BUSINESS AS NAME"] == "GO TAVERN & LIQUORS"]#.drop_duplicates()


,LEGAL NAME,ADDRESS,DOING BUSINESS AS NAME,LICENSE DESCRIPTION,BUSINESS ACTIVITY
21663,"DOROTHY'S LIQUORS, INC.",3219 W ARMITAGE AVE 1,GO TAVERN & LIQUORS,Outdoor Patio,Sale of Liquor Outdoors on Private Property
27284,"DOROTHY'S LIQUORS, INC.",3219 W ARMITAGE AVE 1,GO TAVERN & LIQUORS,Package Goods,Retail Sales of Packaged Liquor
22760,"DOROTHY'S LIQUORS, INC.",3219 W ARMITAGE AVE 1,GO TAVERN & LIQUORS,Retail Food Establishment,Retail Sales of Perishable Foods
33084,"DOROTHY'S LIQUORS, INC.",3219 W ARMITAGE AVE 1,GO TAVERN & LIQUORS,Tavern,Tavern - Consumption of Liquor on Premise
43879,"DOROTHY'S LIQUORS, INC.",3219 W ARMITAGE AVE 1,GO TAVERN & LIQUORS,Tobacco,Retail Sale of Tobacco


In [50]:
df = df1[["ADDRESS", "DOING BUSINESS AS NAME","LICENSE DESCRIPTION", "BUSINESS ACTIVITY"]] \
.sort_values(by=["DOING BUSINESS AS NAME","ADDRESS","LICENSE DESCRIPTION",]).drop_duplicates().dropna()
df["DESCRIPTION_STR"] =  df["LICENSE DESCRIPTION"] + " " + df["BUSINESS ACTIVITY"]

In [51]:
df

,ADDRESS,DOING BUSINESS AS NAME,LICENSE DESCRIPTION,BUSINESS ACTIVITY,DESCRIPTION_STR
46149,1838 N WESTERN AVE 102,"""ART ON YOU"" STUDIO INC.",Limited Business License,Provide Photography or Videography Services,Limited Business License Provide Photography o...
15717,4300-4304 N CENTRAL AVE,"""C"" A CATERING AND EVENT COMPANY",Caterer's Liquor License,Catering of Liquor To Events,Caterer's Liquor License Catering of Liquor To...
25192,4300-4304 N CENTRAL AVE,"""C"" A CATERING AND EVENT COMPANY",Consumption on Premises - Incidental Activity,Consumption of Liquor on Premises,Consumption on Premises - Incidental Activity ...
22536,4300-4304 N CENTRAL AVE,"""C"" A CATERING AND EVENT COMPANY",Retail Food Establishment,Retail Sales of Perishable Foods,Retail Food Establishment Retail Sales of Peri...
50594,151 N MICHIGAN AVE LL C-16,"""M"" BAR",Tavern,Tavern - Consumption of Liquor on Premise,Tavern Tavern - Consumption of Liquor on Premise
...,...,...,...,...,...
52642,33 N CARPENTER ST 1,withlove collective,Limited Business License,Hair Services | Retail Sales of General Mercha...,Limited Business License Hair Services | Retai...
52038,33 N CARPENTER ST 1,withlove collective,Retail Food Establishment,Sale of Food Prepared Onsite Without Dining Area,Retail Food Establishment Sale of Food Prepare...
9235,5307 S HYDE PARK BLVD 12 1206,yah signature llc,Regulated Business License,Online Sales (Home Based Business),Regulated Business License Online Sales (Home ...
10596,633 S PLYMOUTH CT 1,yarnify!,Limited Business License,Retail Sales of General Merchandise,Limited Business License Retail Sales of Gener...


In [78]:
df["LICENSE DESCRIPTION"].value_counts()/len(df)

Limited Business License                               0.264073
Retail Food Establishment                              0.222007
Regulated Business License                             0.162040
Consumption on Premises - Incidental Activity          0.050945
Tobacco                                                0.037331
Motor Vehicle Services License                         0.029248
Peddler License                                        0.025330
Package Goods                                          0.022880
Children's Services Facility License                   0.016880
Public Place of Amusement                              0.016124
Tavern                                                 0.016043
Manufacturing Establishments                           0.014206
Pharmaceutical Representative                          0.013287
Outdoor Patio                                          0.013083
Shared Kitchen User (Long Term)                        0.012593
Commercial Garage                       

In [79]:
df["BUSINESS ACTIVITY"].value_counts()

Retail Sales of Perishable Foods                                                                                                               4463
Consumption of Liquor on Premises                                                                                                              2447
Preparation of Food and Dining on Premise With Seating                                                                                         2114
Sale of Food Prepared Onsite With Dining Area                                                                                                  1341
Retail Sales of General Merchandise                                                                                                            1247
                                                                                                                                               ... 
Electronic Equipment Repair | Payment Center | Retail Sales of Clothing / Accessories / Shoes | Retail Sales of 

In [55]:
# NOTE: keep the index after the groupby. It can relate us back to the base biz name/address of df
byBiz = df.groupby(['ADDRESS', 'DOING BUSINESS AS NAME'])['DESCRIPTION_STR'].transform(lambda x: ' '.join(x))

byBiz = df[['ADDRESS','DOING BUSINESS AS NAME']].merge(byBiz, left_index=True, right_index=True, how="right").drop_duplicates()

byBiz["DESCRIPTION_STR"] = byBiz["DOING BUSINESS AS NAME"] + " " + byBiz["DESCRIPTION_STR"]


In [68]:
# Now we have a dataframe for each unique biz location and all of it's relevant license and biz activity in one column
print(len(byBiz))
byBiz.head()

40643


,ADDRESS,DOING BUSINESS AS NAME,DESCRIPTION_STR
46149,1838 N WESTERN AVE 102,"""ART ON YOU"" STUDIO INC.","""ART ON YOU"" STUDIO INC. Limited Business Lice..."
15717,4300-4304 N CENTRAL AVE,"""C"" A CATERING AND EVENT COMPANY","""C"" A CATERING AND EVENT COMPANY Caterer's Li..."
50594,151 N MICHIGAN AVE LL C-16,"""M"" BAR","""M"" BAR Tavern Tavern - Consumption of Liquor ..."
3795,3515 W 26TH ST 1,"""SABOR MEXICO"" RESTAURANT","""SABOR MEXICO"" RESTAURANT Consumption on Premi..."
9183,1507 E 53RD ST 6 623,"""Savoirfaire""","""Savoirfaire"" Peddler License Combination - St..."


In [96]:
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
import string

def process_text(text):
    """Process text function.
    Input:
        text: a string
    Output:
        text_clean: a list of words containing the processed text

    """
    stemmer = PorterStemmer()
    stopwords_english = stopwords.words('english')
    # to lower
    text = text.lower()
    
    # alphanumeric characters only
    text = re.sub(r'\W+', ' ', text)
    # get rid of numeric values
    #text = ''.join([i for i in text if not i.isdigit()])
    # eliminate extra white space
    text=" ".join(text.split())
    
    # eliminate words/tokens with less than two characters
    #text =[i for i in text.split() if len(i)>=2]
    # get rid of certain words
    #delete_words = ["the", "and", "for", "more", "with", "number", "com", "inc", "www"]
    #text = [i for i in text if i not in delete_words]
    """
    # remove stock market tickers like $GE
    text = re.sub(r'\$\w*', '', text)
    # remove old style retext text "RT"
    text = re.sub(r'^RT[\s]+', '', text)
    # remove hyperlinks    
    text = re.sub(r'https?://[^\s\n\r]+', '', text)
    # remove hashtags
    # only removing the hash # sign from the word
    text = re.sub(r'#', '', text)
    """
    # tokenize texts
    
    #tokenizer = textTokenizer(preserve_case=False, strip_handles=True,
     #                          reduce_len=True)
    text_tokens = word_tokenize(text)

    texts_clean = []
    for word in text_tokens:
        if (word not in stopwords_english and  # remove stopwords
                word not in string.punctuation):  # remove punctuation
            # texts_clean.append(word)
            stem_word = stemmer.stem(word)  # stemming word
            texts_clean.append(stem_word)

    return texts_clean

t = byBiz["DESCRIPTION_STR"][100:150].apply(lambda x: process_text(x))
t

50

## Let's try to Get Some Big Rule-Based Categories


In [108]:
restaurant = ["restaurant"]
tavern = ["tavern", "bar"]
liquor = ["liquor"]
food = ["food"]

rules_cats = []   
for i in t:
# check if any items from list is in the description strings
    if any(item in i for item in restaurant):
        rules_cats.append("restaurant")
    elif any(item in i for item in tavern):
        rules_cats.append("bar")
    elif any(item in i for item in liquor):
        rules_cats.append("liquor")
    elif any(item in i for item in food):
        rules_cats.append("food")
    else:
        rules_cats.append("other")
#pd.DataFrame({"rulebased_cat": rules_cats }     )
pd.concat([byBiz[100:150].reset_index(), pd.DataFrame({"rulebased_cat": rules_cats }     )],axis=1)

,index,ADDRESS,DOING BUSINESS AS NAME,DESCRIPTION_STR,rulebased_cat
0,29497,6774 N NORTHWEST HWY 1 B,1845 ESTHETICS,1845 ESTHETICS Limited Business License Miscel...,other
1,51998,1741 S RUBLE ST,18TH & RUBLE SHELL,18TH & RUBLE SHELL Filling Station Operation o...,food
2,25666,2053-2057 W 18TH ST 1,"18TH STREET CASA DE CULTURA, NFP","18TH STREET CASA DE CULTURA, NFP Limited Busin...",other
3,27195,4117 W OAKTON ST,"1913 N. HALSTED, INC.","1913 N. HALSTED, INC. Regulated Business Licen...",other
4,21471,1060 W ADDISON ST FIELD LEVEL & MEZZ,1914 CLUB,1914 CLUB Consumption on Premises - Incidental...,liquor
5,42123,5000 N KENMORE AVE,1919 HOMES,1919 HOMES Shared Housing Unit Operator Shared...,other
6,30406,167 N GREEN ST 1300,"1989 AVIATION CAPITAL, LLC","1989 AVIATION CAPITAL, LLC Limited Business Li...",other
7,30659,4748 N CUMBERLAND AVE 1ST,"1DOLLAR STORE.COM, INC.","1DOLLAR STORE.COM, INC. Limited Business Licen...",food
8,352,4651 W 79TH ST 2 202,1Infotech,1Infotech Limited Business License Administrat...,other
9,26145,5435 W CONGRESS PKWY 1ST,1MC CLEANING LLC,1MC CLEANING LLC Regulated Business License Re...,other


# Feature Creation

In [ ]:
def build_freqs(tweets, ys):
    """Build frequencies.
    Input:
        tweets: a list of tweets
        ys: an m x 1 array with the sentiment label of each tweet
            (either 0 or 1)
    Output:
        freqs: a dictionary mapping each (word, sentiment) pair to its
        frequency
    """
    # Convert np array to list since zip needs an iterable.
    # The squeeze is necessary or the list ends up with one element.
    # Also note that this is just a NOP if ys is already a list.
    yslist = np.squeeze(ys).tolist()

    # Start with an empty dictionary and populate it by looping over all tweets
    # and over all processed words in each tweet.
    freqs = {}
    for y, tweet in zip(yslist, tweets):
        for word in process_tweet(tweet):
            pair = (word, y)
            if pair in freqs:
                freqs[pair] += 1
            else:
                freqs[pair] = 1

    return freqs